# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/messagerjulien/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [2]:
mySeed = 213

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [3]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

Une méthode simple pour estimer les coefficients serait la méthode des moindres carrés. On l'implémente ci-dessous :

In [4]:
import statsmodels.api as sm

# Add a constant column to the predictors for the intercept term (θ0)
X = sm.add_constant(train_set['inputs'])
y = train_set['targets']

# Create and fit the OLS model
model = sm.OLS(y, X)
results = model.fit()

# Print the estimated coefficients
print(results.params)

[10.61785638  2.07480088  2.04334089  4.31200556]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Pour entraîner un réseau de neurones avec une descente de gradients stochastique, on peut utiliser une architecture simple avec un réseau de neurones à une seule couche. En termes de performance, on évite ainsi un problème de sur-apprentissage qui expliquerait trop bien notre ensemble de coefficients. De plus, on est ici avec un problème linéaire, on peut utiliser une seule couche avec les poids associés aux neurones représentant les coefficients θ1, θ2, θ3 et le biais est représenté par θ0.

In [6]:
import torch.nn as nn
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)



    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [8]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # Zero the gradients from the previous iteration
        optimizer.zero_grad()

        # Forward pass
        outputs = mySimpleNet(batch_inputs)

        # Calculate the loss
        loss = criterion(outputs, batch_targets)

        # Backward pass (calculate gradients)
        loss.backward()

        # Update the model's parameters
        optimizer.step()



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

Les valeurs des coefficients estimées par le modèle sont stockées comme poids et biais. On peut y accéder de la manière suivante :

In [18]:
# Get the weights (θ1, θ2, θ3)
weights = mySimpleNet.linear.weight.data

# Get the bias (θ0)
bias = mySimpleNet.linear.bias.data

# Print the values
print("Weights :", weights)
print("Bias :", bias)

Weights : tensor([[0.0183, 0.0147, 0.0400]])
Bias : tensor([12.7531])


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [17]:
X_test_tensor = torch.tensor(test_set['inputs'], dtype=torch.float32)  # Convert to tensor
predictions_nn = mySimpleNet(X_test_tensor)  # Get predictions

print("Coefficients :", weights, bias)


Coefficients : tensor([[0.0183, 0.0147, 0.0400]]) tensor([12.7531])


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [19]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [23]:
# Nb de paramètres dans self.Down1: (calcul "à la main")
#64 * 3 * 128 + 128  + 2 * 128 = 24576 + 128 + 256 = 24960
# vecteur entrée * taille du noyau * vecteur sortie + biais + 2 * vecteur sortie pour la normalisation des batchs

# Nb de paramètres au total:
from torchsummary import summary
summary(model, (1, 10000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 64, 10000]             256
       BatchNorm1d-2            [-1, 64, 10000]             128
              ReLU-3            [-1, 64, 10000]               0
            Conv1d-4            [-1, 64, 10000]          12,352
       BatchNorm1d-5            [-1, 64, 10000]             128
              ReLU-6            [-1, 64, 10000]               0
Double_conv_causal-7            [-1, 64, 10000]               0
         MaxPool1d-8             [-1, 64, 5000]               0
            Conv1d-9            [-1, 128, 5000]          24,704
      BatchNorm1d-10            [-1, 128, 5000]             256
             ReLU-11            [-1, 128, 5000]               0
           Conv1d-12            [-1, 128, 5000]          49,280
      BatchNorm1d-13            [-1, 128, 5000]             256
             ReLU-14            [-1, 12

La couche self.down1 compte 24960 paramètres. Le réseau compte quant à lui 2872641 paramètres.

**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

La taille du vecteur d'entrée est réduite de différentes manières :


*   Ici on utilise des couches de convolutions avec le paramètre stride > 1, ce qui permet de réduire la dimension des tenseurs. (Applique la couche de convolution ici qu'une fois sur cinq)
*   Avec des couches de Pooling.

Dans la deuxième partie du réseau, la taille est restituée par les couches Up_causal qui rétablissent les pertes des couches de convolution précédentes.





**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Le champ réceptif est augmenté avec chaque couche de convolution mais aussi avec l'utilisation de couches de pooling.

La taille du champ réceptif en sortie est égal à la taille du champ en entrée auquel on rajoute :

In [24]:
champ_r = 3 + (3-1)*2 = 7
#noyau entrée + (taille du noyau - 1) * (coefficient de dilatation)

SyntaxError: cannot assign to expression (<ipython-input-24-237b23b52bb6>, line 1)

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [25]:
model = causalFCN()

# Create two input tensors that differ only in one component
input_tensor1 = torch.zeros(1, 1, 10000)  # All zeros
input_tensor2 = input_tensor1.clone()
input_tensor2[0, 0, 5000] = 1  # Set a single element to 1

# Get the model outputs for both inputs
output1 = model(input_tensor1)
output2 = model(input_tensor2)

# Find the indices where the outputs differ
diff_indices = torch.nonzero(output1 != output2)

# The receptive field is the range of these indices
receptive_field_start = diff_indices[:, 2].min().item()
receptive_field_end = diff_indices[:, 2].max().item()
receptive_field_size = receptive_field_end - receptive_field_start + 1

print("Receptive field start:", receptive_field_start)
print("Receptive field end:", receptive_field_end)
print("Receptive field size:", receptive_field_size)

Receptive field start: 0
Receptive field end: 9999
Receptive field size: 10000


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



Elle ne dépend pas car on utilise ici des convolutions causales (causalFCN) c'est-à-dire qui ne prennent en compte que les valeurs du présent et passé.

\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

Ce que les auteurs appellent "positive samples" dans l'article sont les échantillons où l'on observe l'événement souhaité, ici il s'agit d'échantillons où une personne clique sur une annonce sur un site. "negative samples" à l'inverse sont des échantillons où il n'y a pas le résultat souhaité.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

Les zi sont les échantillons que l'on a en entrée du système et qui représentent les positive et negative samples.

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

Cela est dû à l'utilisation de la fonction log

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?